In [16]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from argparse import ArgumentParser
import argparse
import logging
import logging.config
from selenium import webdriver as wd
import selenium
import numpy as np
import schema
import json
import urllib
import datetime as dt

In [ ]:
driver  = webdriver.chrom('C:\Users\ahmad\Desktop\Internship_Project\Glassdoor_Scraper\chromedriver.exe')

url = 'https://www.glassdoor.ie/Reviews/Aptiv-Reviews-E1891089.htm'

//*[@id="userEmail"]
//*[@id="userPassword"]
//*[@id="InlineLoginModule"]/div/div/div/div[1]/div[3]/form/div[3]/div[1]/button

driver.get(url)

In [15]:
def scrape(field, review, author):

    def scrape_date(review):
        return review.find_element_by_tag_name('time').get_attribute('datetime')

    def scrape_emp_title(review):
        if 'Anonymous Employee' not in review.text:
            try:
                res = author.find_element_by_class_name(
                    'authorJobTitle').text.split('-')[1]
            except Exception:
                logger.warning('Failed to scrape employee_title')
                res = np.nan
        else:
            res = np.nan
        return res

    def scrape_location(review):
        if 'in' in review.text:
            try:
                res = author.find_element_by_class_name(
                    'authorLocation').text
            except Exception:
                res = np.nan
        else:
            res = np.nan
        return res

    def scrape_status(review):
        try:
            res = author.text.split('-')[0]
        except Exception:
            logger.warning('Failed to scrape employee_status')
            res = np.nan
        return res

    def scrape_rev_title(review):
        return review.find_element_by_class_name('summary').text.strip('"')
    
    def scrape_years(review):
        try:
            res = review.find_element_by_class_name('mainText').text.strip('"')
        except Exception:
            res = np.nan
        return res
    
    def scrape_helpful(review):
        try:
            x = review.find_element_by_class_name(
                'helpfulReviews').text.strip('""')
            res = x[x.find("(")+1:x.find(")")]
        except Exception:
            res = 0
        return res

    def scrape_pros(review):
        try:
            res = review.find_elements_by_class_name(
                'v2__EIReviewDetailsV2__fullWidth')[0].find_elements_by_tag_name('p')[1].text
        except Exception:
            res = np.nan
        return res

    def scrape_cons(review):
        try:
            res = review.find_elements_by_class_name(
                'v2__EIReviewDetailsV2__fullWidth')[1].find_elements_by_tag_name('p')[1].text
        except Exception:
            res = np.nan
        return res

    def scrape_advice(review):
        try:
            res = review.find_elements_by_class_name(
                'v2__EIReviewDetailsV2__fullWidth')[2].find_elements_by_tag_name('p')[1].text
        except Exception:
            res = np.nan
        return res

    def scrape_overall_rating(review):
        try:
            ratings = review.find_element_by_class_name('gdStars')
            overall = ratings.find_element_by_class_name(
                'rating').find_element_by_class_name('value-title')
            res = overall.get_attribute('title')
        except Exception:
            res = np.nan
        return res
    
    def _scrape_subrating(i):
        try:
            ratings = review.find_element_by_class_name('gdStars')
            subratings = ratings.find_element_by_class_name(
                'subRatings').find_element_by_tag_name('ul')
            this_one = subratings.find_elements_by_tag_name('li')[i]
            res = this_one.find_element_by_class_name(
                'gdBars').get_attribute('title')
        except Exception:
            res = np.nan
        return res
    
    def scrape_work_life_balance(review):
        return _scrape_subrating(0)

    def scrape_culture_and_values(review):
        return _scrape_subrating(1)

    def scrape_career_opportunities(review):
        return _scrape_subrating(2)

    def scrape_comp_and_benefits(review):
        return _scrape_subrating(3)

    def scrape_senior_management(review):
        return _scrape_subrating(4)

    funcs = [
        scrape_date,
        scrape_emp_title,
        scrape_location,
        scrape_status,
        scrape_rev_title,
        scrape_years,
        scrape_helpful,
        scrape_pros,
        scrape_cons,
        scrape_advice,
        scrape_overall_rating,
        scrape_work_life_balance,
        scrape_culture_and_values,
        scrape_career_opportunities,
        scrape_comp_and_benefits,
        scrape_senior_management
    ]

    fdict = dict((s, f) for (s, f) in zip(SCHEMA, funcs))

    return fdict[field](review)